In [42]:
import numpy as np
import scipy
from scipy.linalg import toeplitz

def postprocess(m, n, b):
    extraction_percent = n / m
    c = np.random.randint(0, 2, m)
    r = np.random.randint(0, 2, n)
    
    T = np.array(toeplitz(c, r))
    out = np.zeros(m)
    running = np.zeros(n)
    
    for i in range(m):
        for j in range(n):
            running[j] = b[j] and T[i, j]
            if j > 0:
                running[j] = bool(running[j-1]) ^ bool(running[j])
            else:
                continue
        out[i] = int(running[n-1])
    
    return out

In [43]:
from scipy.special import erfc
def monobit(b,n):
    b_str = str(b)
    sum = 0
    for x in range(1,len(b_str)):
        if b_str[x] == 1:
            sum += 1
        else:
            sum -= 1
    sobs = abs(sum)/sqrt(n)
    print(erfc(sobs/sqrt(2)))

In [44]:
# Importing the required libraries (install in your environment first)
import numpy as np
import pandas as pd
from math import log2, sqrt

# Read in data from your datafile or the provided datafile in this folder to classify QRNG data. 
# Suggested classification strategies include QRNG vs PRNG, QPU vs Simulator, or by individual QPU

#Sample datafile of QRNG (IBM QPUs) vs PRNG data, 12k lines each. Label 1 is QRNG, label 2 is PRNG
data_filePath = 'QRNGvsPRNG_TrainingData.txt'

rng_td = pd.read_table(data_filePath, sep="\s+", header=None)
rng_td.columns =['binary', 'source']
rng_td_nocat = rng_td.copy(deep='true')
rng_features = pd.DataFrame(rng_td['binary'].apply(list).tolist())
rng_td = pd.concat([rng_td.drop(columns='binary'), rng_features], axis=1)


In [96]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X = rng_td.drop(columns='source').values
y = rng_td['source'].values
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

gb_model = GradientBoostingClassifier(random_state=42)

# Train the model. Define X_train and y_train from your training dataframe using 
# sklearns train_test_split method
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_model.predict(X_test)

# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy: ", accuracy_gb)


Gradient Boosting Accuracy:  0.5214583333333334


In [97]:
data_filePath2 = '100 qubits QRNG Output_Training Data.txt'
rng2_td = pd.read_table(data_filePath2, sep="\s+", header=None)

rng2_features = pd.DataFrame(rng2_td.loc[0].apply(list).tolist())

y_pred2_gb = gb_model.predict(rng2_features.to_numpy())

# Calculate the accuracy of the Gradient Boosting model
accuracy2_gb = accuracy_score(np.ones(100), y_pred2_gb)
print("Gradient Boosting Accuracy: ", accuracy2_gb)

Gradient Boosting Accuracy:  0.54


In [223]:
from qiskit import QuantumCircuit, transpile
HGateCircuit = QuantumCircuit(8)
def transmit(A,GA):
    for i in range(8):
        if(GA[i] == 0): #B[i] = 0, measure in z, B[i] = 1, measure in x
            if A[i] == 0:
                HGateCircuit.h(i);    
#BB84
#a: (4 + δ)n, b: (4 + δ)n
A = np.random.randint(0, 2, 8) #PRNG
GA = np.random.randint(0, 2, 8) #PRNG
B = np.random.randint(0, 2, 8) #PRNG
GB = np.random.randint(0, 2, 8) #PRNG
transmit(A,GA)
transmit(B,GB)


In [139]:
def calculate_max_entropy(bit_string):
    probabilities = np.bincount([int(bit) for bit in bit_string]) / len(bit_string)
    entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
    return entropy

In [140]:
def calculate_min_entropy(bit_string):
    probabilities = np.bincount([int(bit) for bit in bit_string]) / len(bit_string)
    maxp = np.max(probabilities)
    min_entropy = - np.log2(maxp)
    return min_entropy